## Project Requirements:
Scrape profiles and get:
1. the number of posts
2. the follower count
3. number of likes on most liked post
4. numbner of likes

## Follow Up
Save each profile in its own data frame with each row corresponding to a post <br>
export as a excel file with each profile as a worksheet <br>
find a way to edit so that the sheet includes some global account variables (ex. number of followers) outside the dataframe

In [1]:
from instascrape import Profile, scrape_posts, Post
from selenium.webdriver import Chrome
from datetime import datetime
import pandas as pd
import numpy as np
import pickle
from openpyxl.workbook.child import INVALID_TITLE_REGEX
from openpyxl import workbook
import re
from utils import check_valid_username, write_to_excel, get_top_post, create_post_df
from pathlib import Path
from tqdm import tqdm

In [2]:
cwd = Path.cwd()

#mac
#chrome_driver_path = cwd / 'chromedriver'

#windows
chrome_driver_path = cwd / 'chromedriver.exe'
driver = Chrome(chrome_driver_path)


In [3]:
headers = {
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Mobile Safari/537.36 Edg/87.0.664.57",
    "cookie": "sessionid=7320119797%3AiFhQYSC7AxwAcY%3A17"
}

In [4]:
users = pd.read_csv("Social_Media_Study_Handles.csv")
users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts'])

#Specifying datatype for each columns
users['School'] = users['School'].astype(object)
users['IG Username'] = users['IG Username'].astype(object)
users['follower_count'] = users['follower_count'].astype('Int64')
users['number_of_posts'] = users['number_of_posts'].astype('Int64')

# users = users.reindex(columns=[*users.columns, 'follower_count', 'number_of_posts', 'top_post_likes', 
# 'top_post_date_posted', 'top_post_url', 'first_post_likes', 'first_post_date_posted', 'first_post_url'])
# This code is no longer userful since we are storing each account's posts in its own dataframe
# users['top_post_likes'] = users['top_post_likes'].astype('Int64')
# users['top_post_date_posted'] = users['top_post_date_posted'].astype('datetime64[ns]')
# users['top_post_url'] = users['top_post_url'].astype(object)
# users['first_post_likes'] = users['first_post_likes'].astype('Int64')
# users['first_post_date_posted'] = users['first_post_date_posted'].astype('datetime64[ns]')
# users['first_post_url'] = users['first_post_url'].astype(object)

In [33]:
#test purposes
# users = users.head(5).append(users.iloc[89])

#List of dataframes representing the posts of each user
users_df = []

#indicies of all users with more than 200 posts
index_greater_200 = []

#indicies of all users with less than 200 posts
index_200_or_less = []

#Index of null
index_null = []

for i in tqdm(range(0, len(users))):
    #Putting a try catch statement here because there seems to be some errors that could be fixed with an extra try
    attmepts_allowed = 3
    for attmept in range(attmepts_allowed):
        try:
            if (not pd.isnull(users.iloc[i]['IG Username'])):
                #Scrape the profile
                print(users.iloc[i]['IG Username'])
                profile = Profile(users.iloc[i]['IG Username'])
                profile.scrape(headers=headers)

                #Adding profile datapoints to dataframe
                users.at[i, 'follower_count'] = profile.followers
                users.at[i, 'number_of_posts'] = profile.posts

                #Scraping Posts

                posts = profile.get_posts(webdriver=driver)
                print(len(posts))

                #account has less than 200 posts
                if (len(posts) <= 200):
                    scraped_posts, unscraped = scrape_posts(posts, webdriver=driver, silent=False, headers=headers, pause=10)
                    
                    #Appending dataframe from account to each 
                    users_df.append(create_post_df(scraped_posts))
                    index_200_or_less.append(i)
                
                else:
                    print("{} with username {} has {} posts, more than 200".format(users.iloc[i]['School'], users.iloc[i]['IG Username'], len(posts)))
                    
                    #no matter what, we want to keep the dimensions of user_df the same as users
                    users_df.append(create_post_df([]))
                    index_greater_200.append(i)
                        
            # if no username, we're just going to add an empty dataframe
            else:
                users_df.append(create_post_df([]))
                index_null.append(i)

        #Error, let's try again
        except Exception as e:
            print("an error has occured on {}".format(users.iloc[i]['IG Username']))
            print(e)
            #Adding empty dataframe to list anyways just to keep the indicies matching

        #No errors, let's exit the loop
        else:
            break

    #Did we finish the for loop, that means all attempts failed
    else:
        #Adding empty dataframe to list anyways just to keep the indicies matching
        users_df.append(create_post_df([]))
        print("all attempts failed")
    
    #Writing test dataframes to pickle object for easier debugging in the future
    if (i % 10 == 0 and i != 0):
        with open('test_dataframes_{}-{}.obj'.format(i-10, i), 'wb') as f:
            pickle.dump((users_df, users), f)
        
        
        
        #old code from just getting top and first posts
        # top_post = get_top_post(scraped_posts)

        # #Post are scraped from most recent to oldest, therefore, the earliest post will be the last one scraped in theory
        # first_post = scraped_posts[-1]

        # users.at[i, 'top_post_likes'] = top_post.likes
        # users.at[i, 'top_post_date_posted'] = datetime.fromtimestamp(top_post.timestamp)
        # users.at[i, 'top_post_url'] = "instagram.com/p/{}".format(top_post.shortcode)
        # users.at[i, 'first_post_likes'] = first_post.likes
        # users.at[i, 'first_post_date_posted'] = datetime.fromtimestamp(first_post.timestamp)
        # users.at[i, 'first_post_url'] = "instagram.com/p/{}".format(first_post.shortcode)


  0%|          | 0/121 [00:00<?, ?it/s]

willseyeresidents
33
0: CDABF2hhEhP - 2020-07-23 17:49:11
1: CC6_LwDhnTu - 2020-07-21 18:56:19
2: CC4Kk8ZBGIc - 2020-07-20 16:38:09
3: CCtc94uh_SW - 2020-07-16 12:47:12
4: CCqrp49B5j1 - 2020-07-15 10:57:49
5: CCjOYsGBwWO - 2020-07-12 13:27:23
6: CCecas4hdY- - 2020-07-10 16:53:47
7: CCHSdAehmf7 - 2020-07-01 17:04:11
8: CB3xPaehw43 - 2020-06-25 16:25:22
9: CBy5lzbBlq8 - 2020-06-23 19:02:08
10: CBwQp8qhq3y - 2020-06-22 18:25:57
11: CBtTXidBM4k - 2020-06-21 14:51:56
12: CBqW91ohOfD - 2020-06-20 11:25:39
13: CBoCRLXhQex - 2020-06-19 13:46:18
14: CBlvolqAn4t - 2020-06-18 16:25:00
15: CBjLzmHBJxE - 2020-06-17 16:33:27
16: CANgGIPAQc0 - 2020-05-15 09:56:03
17: CALFcpLAZq1 - 2020-05-14 11:24:43
18: B_lFRQnBz3j - 2020-04-29 17:12:01
19: B-sAg1yhXLQ - 2020-04-07 13:13:45
20: B9Z5wFmBtvD - 2020-03-06 14:56:56
an error has occured on willseyeresidents
list index out of range
willseyeresidents
33
0: CDABF2hhEhP - 2020-07-23 17:49:11
1: CC6_LwDhnTu - 2020-07-21 18:56:19
2: CC4Kk8ZBGIc - 2020-07-20 16

  1%|          | 1/121 [09:49<19:39:36, 589.80s/it]

32: B4BZYIpAuYT - 2019-10-24 20:55:18
bascompalmereye


  2%|▏         | 2/121 [10:25<8:42:50, 263.61s/it] 

...

an error has occured on ketteringhealth
Cannot assign nan to integer series
all attempts failed
pcomeducation
an error has occured on pcomeducation
Cannot assign nan to integer series
pcomeducation
an error has occured on pcomeducation
Cannot assign nan to integer series
pcomeducation


 99%|█████████▉| 120/121 [7:22:37<00:02,  2.56s/it]

an error has occured on pcomeducation
Cannot assign nan to integer series
all attempts failed
stjohnseh
an error has occured on stjohnseh
Cannot assign nan to integer series
stjohnseh
an error has occured on stjohnseh
Cannot assign nan to integer series
stjohnseh


100%|██████████| 121/121 [7:22:39<00:00, 219.50s/it]

an error has occured on stjohnseh
Cannot assign nan to integer series
all attempts failed


In [35]:
#Write dataframes to a pickle object
with open('less_than_200_dataframes.obj', 'wb') as f:
    pickle.dump((users_df, users), f)

In [72]:
users_df_np = np.array(users_df, dtype=object)
write_to_excel(users_df_np[index_200_or_less], users.iloc[index_200_or_less].reset_index())

TJU/Wills Eye Hospital
Johns Hopkins/Wilmer
University of Iowa
Mass. Eye and Ear
UCLA/Doheny
Duke
University of Utah/Moran
University of California San Francisco (UCSF)
UTSW
Emory
University of Pennsylvania/Scheie
Washington University in Saint Louis
New York Eye and Ear
University of Illinois at Chicago
New York Presbyterian (Columbia)
Vanderbilt
University of Colorado
NYU
University of Alabama


C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: Us

Ohio State
UC Davis
UPMC
UC Irvine
Wake Forest
New York Presbyterian (Cornell)
Northwestern
UNC
Rush
Indiana University
Montefiore/Albert Einstein
Loyola
Detroit Medical Center/Wayne State University
MUSC
Yale
University of Maryland
University of Kentucky
MedStar/Georgetown
SUNY Downstate
Tulane
Loma Linda
West Virginia University
University of Chicago
University of Rochester
University of Tennesee
University of Virginia
University of Louisville
UMKC
LSU Shreveport
UM Columbia
Brown
Cook County Health and Hospitals System
University of South Florida
University of Kansas
University of Texas/Methodist Hospital (Houston)
Albany
University of Arkansas
Saint Louis University School of Medicine
University at Buffalo
EVMS
Medical College of Georgia
Beaumont Health (Taylor)


C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
C:\Users\Albert\miniconda3\envs\instagramScraper\lib\site-packages\openpyxl\workbook\child.py:99: Us

In [71]:
test_df = users.iloc[index_200_or_less].reset_index
test_df

<bound method DataFrame.reset_index of      index                                     School  \
0        0                     TJU/Wills Eye Hospital   
2        2                       Johns Hopkins/Wilmer   
3        3                         University of Iowa   
4        4                          Mass. Eye and Ear   
5        5                                UCLA/Doheny   
..     ...                                        ...   
99      99  Saint Louis University School of Medicine   
102    102                      University at Buffalo   
107    107                                       EVMS   
111    111                 Medical College of Georgia   
117    117                   Beaumont Health (Taylor)   

                       IG Username  follower_count  number_of_posts  
0                willseyeresidents            1984               33  
2               wilmereyeresidents            1511               34  
3                         uiowaeye            1482              13

In [59]:
users.loc[pd.isna(users["IG Username"])].index.tolist()

[83, 89, 95, 100, 104, 105, 108, 110, 112]

In [42]:
test_list = index_200_or_less

In [61]:
users.iloc[index_200_or_less]

,index,School,IG Username,follower_count,number_of_posts
0,0,TJU/Wills Eye Hospital,willseyeresidents,1984,33
2,2,Johns Hopkins/Wilmer,wilmereyeresidents,1511,34
3,3,University of Iowa,uiowaeye,1482,131
4,4,Mass. Eye and Ear,harvardophthalmologyresidents,1571,186
5,5,UCLA/Doheny,steineyeresidents,1043,49
...,...,...,...,...,...
99,99,Saint Louis University School of Medicine,slu_som,1116,90
102,102,University at Buffalo,buffaloophthalmology,1009,26
107,107,EVMS,evms_eye,140,8
111,111,Medical College of Georgia,mcg_aug,465,78


In [59]:
def write_to_excel(user_post_dfs, users_df):
    if (len(user_post_dfs) != users_df.shape[0]):
        raise ValueError("length of the posts dataframes and user dataframes are mismatched, there are {} posts dataframes and {} rows in the user dataframe".format(
            len(user_post_dfs), users_df.shape[0]))

    with pd.ExcelWriter('User Data.xlsx', engine='openpyxl') as writer:
        for i in users_df.index:
            # this regex statement replaces excel's invalid characters with and underscore
            print(users_df.at[i, 'School'])
            title = re.sub(INVALID_TITLE_REGEX, '_', users_df.at[i, 'School'])
            users_df.iloc[[i]].to_excel(writer, sheet_name=title, index=False)
            user_post_dfs[i].to_excel(
                writer, startrow=3, sheet_name=title, index=False)

In [13]:
#Writing test dataframes to pickle object for easier debugging in the future
with open('test_dataframes.obj', 'wb') as f:
    pickle.dump((users_df, users), f)

In [37]:
with open('test_dataframes_110-120.obj', 'rb') as f:
    loaded = pickle.load(f)

users_df = loaded[0]
users = loaded[1]
users = users.reset_index()

In [39]:
len(users_df)

121

In [40]:
users.shape

(121, 5)

In [47]:
index_200_or_less = users.loc[users['number_of_posts'] <= 200].index.tolist()